# Tarea 3: Derivación automática 1

Fecha de envío del PR inicial: **lunes 27 de marzo, antes de la clase**

Fecha de aceptación del PR: **martes 4 de abril, antes de la clase**

## Ejercicio 1

Implementen una nueva estructura paramétrica (`type`) que llamaremos `Dual` y que defina los duales, donde el parámetro debe ser un subtipo de `Real`; la siguiente celda sirve para empezar. La parte que identifica a $f_0$ será llamada `fun`, y la correspondiente a $f'_0$ será `der`.

La definición debe incluir métodos que sean compatibles con las dos propiedades arriba mencionadas, es decir, que el dual de una constante (cualquier número real) sea $(c,0)$, y que el de la variable independiente sea $(x_0,1)$. Para lo segundo definiremos una función `xdual` con la propiedad mencionada.

---

In [1]:
"""
    Dual{T<:Real}

Definición de los duales, donde lo campos son:
...
"""
type Dual{T<:Real}
    fun::T
    der::T
end

Dual

In [2]:
#= 
Definan un método que permita la promoción automática cuando 
las entradas para definir el dual no son del mismo tipo
=#
"""
De ésta manera se promueven los duales para que sean del mismo tipo
"""
Dual(a,b) = Dual(promote(a,b)...)

Dual

In [3]:
#= 
Aquí se define la función `xdual`, que se usará para identificar 
la variable independiente. La función dependerá de x_0, y debe 
regresar el Dual apropiado a la variable independiente
=#
"""
xdual(x0) -> Dual(x0, 1)

Función que regresa el Dual(el vector que tiene como primera entrada f(x0) y segunda entrada f'(x0)) del número x0. 
"""
xdual(x0)=Dual(x0,1)

xdual

In [4]:
xdual(1)

Dual{Int64}(1,1)

In [5]:
#= 
Aquí se define un método que garantiza que el dual de una constante 
(número) cumple lo requerido
=#
"""
Dual(x0)

Función que regresa el Dual de una constante Dual(x0,0). 
"""
Dual(x0)=Dual(x0,0)

Dual

In [6]:
Dual(2)

Dual{Int64}(2,0)

In [7]:
#= 
Muestren que su código funciona con tests adecuados para crear duales,
para promoverlos, y al definir el dual de un número y `xdual`.

En esto es útil usar la infraestructura de Julia; ver:
https://julia.readthedocs.io/en/stable/stdlib/test/=#

using Base.Test
a = Dual(1, 2.0)
@testset "pruebas de las componentes de un Dual"begin
    @test a.fun == 1.0
    @test a.der == 2
end

Test Summary:                         | Pass  Total
  pruebas de las componentes de un Dual |    2      2


Base.Test.DefaultTestSet("pruebas de las componentes de un Dual",Any[],2,false)

## Ejercicio 2

Implementen *todas* las operaciones aritméticas definidas arriba. Estas operaciones deben incluir las operaciones aritméticas que involucran un número cualquiera (`a :: Real`) y un dual (`b::Dual`), como por ejemplo `a+b` o `b+a`, etc. Esto se puede hacer implementando los métodos específicos para estos casos (¡y que sirven en cualquier órden!). 

Implementen también la comparación entre duales (`==`). 

Incluyan tests que muestren que cada una de ellas está bien definida, y que sus resultados dan valores consistentes.

**Derivada de la suma y resta de funciones**
$$f(x)=g(x)\pm h(x)\hspace{1cm}\frac{d}{dx}f(x)=f'(x)=g'(x)\pm h'(x)$$

**Derivada de un producto de funciones**
$$f(x)=g(x)h(x)\hspace{1cm}\frac{d}{dx}f(x)=f'(x)=g'(x)h(x)+g(x)h'(x)$$

**Derivada de un cociente**
$$f(x)=\frac{g(x)}{h(x)}\hspace{1cm}\frac{d}{dx}f(x)=f'(x)=\frac{g'(x)h(x)-g(x)h'(x)}{h^2(x)}$$

**Derivada de una función elevada a una potencia real**
$$f(x)=g(x)^\alpha\hspace{1cm}\frac{d}{dx}f(x)=f'(x)=\alpha g(x)^{\alpha-1}g'(x)$$

In [8]:
import Base: +, -, *, /, ^, ==

#= 
Aquí se implementan los métodos necesarios para cada función; 
en el caso de ^ por ahora nos conformaremos con que la potencia 
sea entera.
=#
#import Base.+
+(a::Dual, b::Dual) = Dual( a.fun+b.fun, a.der+b.der )
+(a::Dual, b::Real) = Dual( a.fun+b, a.der )
+(a::Real, b::Dual) = Dual( a+b.fun, b.der )

#import Base.-
-(a::Dual, b::Dual) = Dual( a.fun-b.fun, a.der-b.der )
-(a::Dual, b::Real) = Dual( a.fun-b, a.der )
-(a::Real, b::Dual) = Dual( a-b.fun, -b.der )

#import Base.*
*(a::Dual, b::Dual) = Dual( a.fun*b.fun, a.der*b.fun+a.fun*b.der )
*(a::Real, b::Dual) = Dual( a*b.fun, a*b.der )
*(a::Dual, b::Real) = Dual( b*a.fun, b*a.der )

#import Base./
/(a::Dual, b::Dual) = Dual( a.fun/b.fun, (a.der*b.fun-a.fun*b.der)/(a.fun*a.fun) )
/(a::Dual, b::Real) = Dual( a.fun/b, a.der/b )
#/(a::Real, b::Dual) = Dual( a.fun/b.fun, (a.der*b.fun-a.fun*b.der)/(a.fun*a.fun) )

#import Base.^
^(a::Dual, b::Int64) = Dual( a.fun^b, b*a.fun^(b-1)*a.der )

#import Base.==
==(a::Dual, b::Dual) =a.fun==b.fun && a.der==b.der
==(a::Real, b::Dual) =a==b.fun && 0.0==b.der
==(a::Dual, b::Real) =a.fun==b && a.der==0.0

== (generic function with 121 methods)

In [9]:
f(x)=2x^3-2

f (generic function with 1 method)

In [10]:
f(xdual(1))

Dual{Int64}(0,6)

In [11]:
# Aquí se incluyen las pruebas necesarias
@testset "pruebas con operaciones entre Duales"begin
    @test Dual(2) == Dual(2,0)
    @test Dual(5.0) == Dual(5.0,0.0)
    @test xdual(1)==Dual(1,1)
    @test xdual(3.0)==Dual(3,1)
    @test Dual(5)==Dual(5,0)
    @test Dual(1)==Dual(1,0)
    @test Dual(1,2)-3==Dual(-2,2)
    @test 3-Dual(1,2)==Dual(2,-2)
    @test Dual(1,2)+3==Dual(4,2)
    @test 3+Dual(1,2)==Dual(4,2)
    @test 3*Dual(1,2)==Dual(3,6)
    @test Dual(1,2)*3==Dual(3,6)
    @test Dual(1,2)/3==Dual(1/3,2/3)
    @test (Dual(2,1))^3==Dual(8,12)
    @test f(xdual(1))==Dual(0,6)
    @test f(xdual(3))==Dual(52,54)
end

Test Summary:                        | Pass  Total
  pruebas con operaciones entre Duales |   16     16


Base.Test.DefaultTestSet("pruebas con operaciones entre Duales",Any[],16,false)

---

## Ejercicio 3

Para reutilizar el código que han hecho en este notebook, y de hecho seguirlo desarrollando, conviene ponerlo dentro de un módulo. Para hacer esto, deberán copiar todo el código necesario (y que aparece en la resolución de los ejercicios anteriores) en un archivo cuyo nombre será "AutomDiff.jl" y cuya estructura será la siguiente

```julia
#=
Aquí viene una explicación de lo que se hace en el módulo, 
los autores y la fecha
=#

# La siguiente instrucción sirve para *precompilar* el módulo
__precompile__(true)

module AD
    import Base: +, -, *, /, ^, ==
    
    export Dual, xdual
    
    # Aquí viene TODO el código que implementaron.
    # Primero uno incluye la definición de Dual y
    # después las operaciones con Duales.
    ...

end
```

Todas las pruebas deberán ser incluidas en un archivo separado llamado "runtest.jl", y su estructura es:

```julia
# Este archivo incluye los tests del módulo AD
include("AutomDiff.jl")
using Base.test
using AD

# A continuación vienen los tests que implementaron y que deben 
# ser suficientemente exhaustivos
...

```

Estos dos archivos deben incluirlos en el envío de su tarea (además del archivo `Tarea3.ipynb`).